<a href="https://colab.research.google.com/github/anuragsaraf1912/Babbar450/blob/master/getMarketData.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import yfinance as yf
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from IPython.display import clear_output

In [53]:
def plotCandleStick(data):
  """Plots candlestick pattern using the Plotly library"""

  candlestick = go.Candlestick(
    x=data["Date"],
    open=data["Open"],
    high=data["High"],
    low=data["Low"],
    close=data["Close"],
    name="Price",
    increasing_line_color='green',
    decreasing_line_color='red'
  )

  # Create volume trace
  volume = go.Bar(
      x=data["Date"],
      y=data["Volume"],
      name="Volume",
      marker=dict(color='rgba(0, 0, 255, 0.3)')
  )

  # Create figure

  fig = make_subplots(specs=[[{"secondary_y": True}]])
  fig.add_trace(
      candlestick,
      secondary_y=False,
  )

  fig.add_trace(
      volume,
      secondary_y=True,
  )

  # Update layout
  fig.update_layout(
      title="Candlestick Chart with Volume",
      height = 1000,
      width = 2300,
      xaxis=dict(
          title="Date",
      ),
  )

  # Configure the y-axis for volume
  fig.update_yaxes(title_text="Price", secondary_y=False, )
  fig.update_yaxes(title_text="Volume", secondary_y=True, range =[0, 5*max(data["Volume"])])

  # Show figure
  fig.show()

In [54]:
import ipywidgets as widgets
from ipywidgets import HBox, VBox

def getTicker(tick):
  if tick == '^NSEI':
    return '^NSEI'
  else:
    return tick + ".NS"

def on_value_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        startDate = start_date_picker.value
        endDate = end_date_picker.value
        ticker = getTicker(dropdown.value)
        if startDate and endDate and ticker:
          global df
          df = (yf.download(ticker, startDate, endDate)).reset_index()
          df['Daily Returns'] = 100*np.log(df['Adj Close']/df['Adj Close'].shift(1))
          df.dropna(inplace=True)
          clear_output(wait = True)
          display(widgets.HBox([dropdown, start_date_picker, end_date_picker])
)

# Date Picker Widgets
start_date_picker = widgets.DatePicker(description='Start Date:')
end_date_picker = widgets.DatePicker(description='End Date:')

# Text Entry Widget
options = ['HDFCBANK', '^NSEI']

# Create the dropdown widget
dropdown = widgets.Dropdown(
    options=options,
    description='Select stock:'
)

# Observing changes
start_date_picker.observe(on_value_change)
end_date_picker.observe(on_value_change)
dropdown.observe(on_value_change)

# Displaying widgets
widgets.HBox([dropdown, start_date_picker, end_date_picker])
#Some stocks to look into: HDFCBANK, WSTCSTPAPR

In [55]:
plotCandleStick(df)

In [38]:
df['Daily Returns'].describe()

,Daily Returns
count,182.000000
mean,0.102041
std,0.906585
min,-6.112420
25%,-0.189523
50%,0.139788
75%,0.522364
max,3.307130


In [44]:
# Create a histogram
fig = go.Figure(data=[go.Histogram(x=df['Daily Returns'])])

# Update layout for better appearance
fig.update_layout(
    title="Daily Returns Distribution",
    xaxis_title="Returns",
    yaxis_title="Count",
    bargap=0.1,  # Gap between bars
    bargroupgap=0.1,  # Gap between groups of bars
    height = 500,
    width = 1000
)

# Show the figure (interactive)
fig.show()

In [36]:
df.tail()

,Date,Open,High,Low,Close,Adj Close,Volume,Daily Returns
178,2024-09-23,25872.550781,25956.000000,25847.349609,25939.050781,25939.050781,209200,0.005726
179,2024-09-24,25921.449219,26011.550781,25886.849609,25940.400391,25940.400391,384100,0.000052
180,2024-09-25,25899.449219,26032.800781,25871.349609,26004.150391,26004.150391,278500,0.002455
181,2024-09-26,26005.400391,26250.900391,25998.400391,26216.050781,26216.050781,370900,0.008116
182,2024-09-27,26248.250000,26277.349609,26151.400391,26178.949219,26178.949219,490300,-0.001416
